In [1]:
from llama_cpp import Llama
import pandas as pd
import os
from tqdm import tqdm

In [2]:
data=pd.read_feather('/Volumes/T7/chroniclingamerica/american-stories/kkk-revival.feather')

In [5]:
data.dtypes

index                     object
headline                  object
newspaper_name            object
date              datetime64[ns]
article                   object
dtype: object

In [17]:
model_path='../../../data/kkk/llm/gpt-oss-20b-F16.gguf'
#https://huggingface.co/unsloth/gpt-oss-20b-GGUF

In [56]:
llm=Llama(model_path=model_path,
                    n_gpu_layers=-1, #Use Apple silicon GPU
                    chat_format='llama-2', #To make the model chat like Llama-2
                    verbose=False,
                    max_tokens=2048,
                    n_ctx=2048,
                    temperature=0.1) #Lower temperature = more deterministic

llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

In [57]:
corrected_article=[]
for article in data['article'].to_numpy()[:10]:
    correction=llm.create_chat_completion(
        messages=[
            {"role": "system",
            "content": """
                You will be given an article from a newspaper published between 1915 and 1944.
                Because of damaged papers and poor scanning quality, the text may contain OCR errors.
                Correct the spelling and grammar errors in the text without changing the meaning.
                """},
            {"role": "user",
            "content": article}
        ],
        # max_tokens=4096,
        temperature=0.1,
    )
    corrected_article.append(correction['choices'][0]['message']['content'])
    break

In [58]:
corrected_article

[" The correct?\n\nMembers of the Y.M.T.A. and B. society possessing musical talent, and there are not a few, are planning to form an orchestra. President W. J. Kerin has the matter in hand, and those anxious to display their ability as musicians should communicate with him. Some of the society's talent, which has thus far signified the intention of joining the proposed orchestra, includes Edward J. Lynch, lead CT. and cornet player; Thomas Mack, cornet; James J. Bonney, Frank May, John Jacobs, John Hannon, Edward Sullivan, James Mccarthy, and Charles Dunn violinists; Henry Kane, John Maerz, Anthony Contoise, and George H. Lynch clarionet; William Lowe and Thomas Lowe, flute; Joseph Boskowski, flute and piccolo; Peter McCrann, drum; James Keegan, bass drum.\n\nThe correct?\n\nMembers of the Y.M.T.A. and B. society possessing musical talent, and there are not a few, are planning to form an orchestra. President W. J. Kerin has the matter in hand, and those anxious to display their abilit

In [42]:
data['article'][0]

'Members of the Y. M. T. A, & B. society possessing musical talent, and there are not a few, are planning to form an orchestra. President w. J Kerin has the matter in hand and those anxious to display their ability as musicians should communicate with him. Some of the society talent which has thus far signified inten- tion of Joining the proposed orches- tra includes Edward J. Lynch, lead- CT. and cornet player; Thomas Mack, cornet; James J. Bonney, Frank May, John Jacobs, John Hannon, Edward Sullivan James Mccarthy and Charles Dunn violinists; Henry Kane, John Maerz, Anthony Contoise and George H. Lynch clarionet; William Lowe and Thomas Lowe, flute; Joseph Bos kowski, flute and piccolo; Peter McCrann, drum; James Keegan, bass drum.'